# Chapter 8

Pairs are probably one of the most important type in your Michelson tool belt. Every contract starts with a pair, every contract ends with a pair. They are a fundamental component of Michelson contracts. A pair is the smallest type of value you can use to store multiple values: it can hold two values of different types. You can also *nest* pairs: a pair can consist of a simple value and another pair or even two other pairs! This model allows nesting multiple pairs one into the other, like Russian dolls.

Along the previous lessons, we encountered a couple of instruction that act on pairs, namely **`CAR`** and **`CDR`**. If you remember, these two opcodes extract the left side and the right side of the pair, respectively. There is also **`PAIR`** that creates a new pair with two provided values. But that's just the tip of the iceberg! There are other instructions you can use to create or manipulate pairs. By the end of this chapter, pairs will have no secret for you!

## Overview of the pair type

A pair is fundamentally a value that holds two other values. These two values can be of any type. In the documentation, you will find pair type written this way: `(pair type1 type2)`. The value of the pair is written this way: `(Pair value1 value2)`. Every smart contract written in Michelson starts and ends with a pair:

In [1]:
storage (pair int int) ;
parameter unit ;
code {
    DROP ;
    PUSH int 6 ;
    PUSH int 7 ;
    PAIR ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit (Pair 0 0) ;

storage (pair int int);
parameter unit;
code { DROP ; PUSH int 6 ; PUSH int 7 ; PAIR ; NIL operation ; PAIR };
RUN: use %default; drop all; push (Unit, (0, 0));
DROP: pop (Unit, (0, 0));
PUSH: push 6;
PUSH: push 7;
PAIR: pop 7, 6; push (7, 6);
NIL: push [];
PAIR: pop [], (7, 6); push ([], (7, 6));

value,type
Pair 7 6,pair int int


You can choose the types you want in a pair, even other pairs:

In [2]:
storage unit ;
parameter unit ;
BEGIN Unit Unit ;
DROP ;
PUSH nat 5 ;
PUSH string "hello" ;
PAIR @first_pair ;
PUSH mutez 5000 ;
NOW ;
PAIR @second_pair ;
PUSH (pair int string) (Pair 3 "hello") ;
PUSH (pair nat mutez) (Pair 5 45678) ;
PAIR @third_pair ;
DUMP ;

storage unit;
parameter unit;
BEGIN: use %default; drop all; push (Unit, Unit);
DROP: pop (Unit, Unit);
PUSH: push 5;
PUSH: push hello;
PAIR: pop hello, 5; push ('hello', 5);
PUSH: push 5000;
NOW: push 1593416783;
PAIR: pop 1593416783, 5000; push (1593416783, 5000);
PUSH: push (3, 'hello');
PUSH: push (5, 45678);
PAIR: pop (5, 45678), (3, 'hello'); push ((5, 45678), (3, 'hello'));

value,type,name
"Pair (Pair 5 45678) (Pair 3 ""hello"")",pair (pair nat mutez) (pair int string),@third_pair
Pair 1593416783 5000,pair timestamp mutez,@second_pair
"Pair ""hello"" 5",pair string nat,@first_pair


## Operations on pairs

A few instructions are available on pairs. The first one is an operation we have already been using since the beginning: **`PAIR`**. **`PAIR`** allows you to take the two elements on top of the stack and put them in a `pair`. It goes without saying that **`PAIR`** only works if there are currently two elements on top of the stack. Unlike `list`, you don't have to specify the type of the two elements, the first element will be on the left side of the pair and the second element on the right side. The instruction was illustrated above.

Next come two other instructions that we have also been using for a while: **`CAR`** and **`CDR`**. If you remember, **`CAR`** extracts the left side of a pair while **`CDR`** extracts the right side:

In [4]:
DROP_ALL ;
PUSH (pair nat string) (Pair 5 "tezos") ;
DUP ;
CAR @left_side ;
SWAP ;
CDR @right_side ;
DUMP ;

DROP_ALL: drop all;
PUSH: push (5, 'tezos');
DUP: push (5, 'tezos');
CAR: pop (5, 'tezos'); push 5;
SWAP: pop 5, (5, 'tezos'); push 5; push (5, 'tezos');
CDR: pop (5, 'tezos'); push tezos;

value,type,name
"""tezos""",string,@right_side
5,nat,@left_side


You should be familiar with their functioning now.  
It is possible to "combine" these two instructions when working with nested pairs (i.e pairs inside pairs). If you have ever checked Michelson contracts, you probably noticed that nested pairs are pretty common. Let's observe the snippet below:

In [7]:
DROP_ALL ;
PUSH (pair (pair nat nat) (pair string string)) (Pair (Pair 5 6) (Pair "hello" "world")) ;
CADR ;
DUMP ;

DROP_ALL: drop all;
PUSH: push ((5, 6), ('hello', 'world'));
CAR: pop ((5, 6), ('hello', 'world')); push (5, 6);
CDR: pop (5, 6); push 6;

value,type
6,nat


This is one long instruction! It created nested pairs: a root pair that contains two other pairs: on the left side, a pair with a `nat` on the left and a `nat` on the right, on the right side, a pair with a `string` on the left and a `string` on the right. As you can see from the result, the **`CADR`** instruction is equal to **`CAR ; CDR ;`**, i.e it will extract the pair on the left side of a pair and then extract the right side of the nested pair. If you want to extract the left side of the nested pair, you can use **`CAAR`**:

In [8]:
DROP_ALL ;
PUSH (pair (pair nat nat) (pair string string)) (Pair (Pair 5 6) (Pair "hello" "world")) ;
CAAR ;
DUMP ;

DROP_ALL: drop all;
PUSH: push ((5, 6), ('hello', 'world'));
CAR: pop ((5, 6), ('hello', 'world')); push (5, 6);
CAR: pop (5, 6); push 5;

value,type
5,nat


To work on the right side of the root pair, you can use **`CDAR`** (to extract its left side) or **`CDDR`** (to extract its right side):

In [9]:
DROP_ALL ;
PUSH (pair (pair nat nat) (pair string string)) (Pair (Pair 5 6) (Pair "hello" "world")) ;
DUP;
CDAR ;
SWAP ;
CDDR ;
DUMP ;

DROP_ALL: drop all;
PUSH: push ((5, 6), ('hello', 'world'));
DUP: push ((5, 6), ('hello', 'world'));
CDR: pop ((5, 6), ('hello', 'world')); push ('hello', 'world');
CAR: pop ('hello', 'world'); push hello;
SWAP: pop hello, ((5, 6), ('hello', 'world')); push hello; push ((5, 6), ('hello', 'world'));
CDR: pop ((5, 6), ('hello', 'world')); push ('hello', 'world');
CDR: pop ('hello', 'world'); push world;

value,type
"""world""",string
"""hello""",string


We can go even further and imagine a three-level nested pair. According to the level you are trying to reach, you can put more **A**s or more **D**s between **C** and **R** to access the value you want:

In [10]:
DROP_ALL ;
PUSH (pair (pair nat nat) (pair int (pair string string))) (Pair (Pair 5 6) (Pair 10 (Pair "hello" "world"))) ;
CDDDR ;
DUMP ;

DROP_ALL: drop all;
PUSH: push ((5, 6), (10, ('hello', 'world')));
CDR: pop ((5, 6), (10, ('hello', 'world'))); push (10, ('hello', 'world'));
CDR: pop (10, ('hello', 'world')); push ('hello', 'world');
CDR: pop ('hello', 'world'); push world;

value,type
"""world""",string
